In [267]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [268]:
def parser(url):
    headers = headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = requests.get(url, headers).content
    soup = BeautifulSoup(content, "html.parser")

    return soup

In [269]:
def job_card(url):
    soup = parser(url)
    
    efc_jobcard = soup.find('div', {'class': 'd-flex flex-column jobs-container ng-star-inserted'}).find_all('efc-job-card')

    card_details = [card.find('efc-card-details') for card in efc_jobcard]
    card_footer = [card.find('efc-card-footer') for card in efc_jobcard]
        
    return card_details, card_footer

In [270]:
efc_url = "https://www.efinancialcareers.co.uk/jobs/in-United-Kingdom" 

jcard = job_card(efc_url)
card_det, card_ftr = jcard[0], jcard[1]


In [274]:

job_title, company, job_link, location, position_type, work_arrangement, salary, posted_on = [], [], [], [], [], [], [], []

span_len, span_txt = [], []


job_dct = {}


for item in card_det:
    job_title.append(item.find('a').get('title'))
    company.append(item.find_all('div')[3].text.lstrip().rstrip())
    job_link.append(item.find('a').get('href'))
    location.append(item.find('span' , {'class': 'dot-divider'}).text)
    position_type.append(item.find('span' , {'class': 'dot-divider ng-star-inserted'}).text)

cnt = 0
for item in card_ftr:
    cnt += 1
    #try:
    work_arrangement.append(item.find('span').text)
    
    ln = len(item.find_all('div'))

    # get span_len  - len(item.find_all('span'))
    # if len = 3 or 2 --> wrk_arr = item.find_all('span')[0].text    salary = item.find_all('span')[1].text
    # if len = 2 --> wrk_arr = item.find_all('span')[0].text    salary = none
    
    span_len.append(ln)
    
 
#job_dct = {'Test': company}
print(job_title)


#job_dct = {"Position": job_title, "Company": company, "Link": job_link, "Location": location, "Position Type": position_type}
#final_df = pd.DataFrame(job_dct)
#print(final_df)



[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [56]:
job_title = [item.find('a').get('title')  for item in cdets][0]
company = [item.find('div', {'class': 'font-body-3 company col ng-star-inserted'}).get_text().lstrip() for item in cdets][0]
job_link = [item.find('a').get('href')  for item in cdets][0]
location = [item.find('span', {'class': 'dot-divider'}).get_text() for item in cdets][0]
pos_type = [item.find('span', {'class': 'dot-divider ng-star-inserted'}).get_text() for item in cdets][0]

work_arr = [item.find('span', {'class': 'dot-divider-after ng-star-inserted'}).text for item in cfter][0]
salary = [item.find('span', {'class': 'dot-divider-after last-job-criteria ng-star-inserted'}).text for item in cfter][0]

posted_on = [item.find('span', {'class': 'font-helper-text dot-divider color-font-secondary ng-star-inserted'}).text.lstrip() for item in cfter][0]



print(job_position)
print(company)
print(job_link)
print(location)
print(pos_type)

print(work_arr)
print(salary)

print(posted_on)

SPV Accountant
Cameron Kennedy 
https://www.efinancialcareers.co.uk/jobs-UK-London-SPV_Accountant.id21200880
London, United Kingdom
Contract
Hybrid
GBP75000 - GBP85000 per annum + Bonus Benefits
13 minutes ago 
